# LangChain 的 Redis 缓存

本笔记本演示如何使用 langchain-redis 包中的 `RedisCache` 和 `RedisSemanticCache` 类来为 LLM 响应实现缓存。

## 设置

首先，安装所需的依赖项并确保 Redis 实例正在运行。

In [ ]:
%pip install -U langchain-core langchain-redis langchain-openai redis

确保 Redis 服务器正在运行。你可以使用 Docker 启动一个：

```
docker run -d -p 6379:6379 redis:latest
```

或者根据你操作系统的说明在本地安装并运行 Redis。

In [ ]:
import os

# 如果设置了环境变量则使用该变量，否则默认为 localhost
REDIS_URL = os.getenv("REDIS_URL", "redis://localhost:6379")
print(f"连接到 Redis：{REDIS_URL}")

Connecting to Redis at: redis://redis:6379


## 导入所需库

In [3]:
import time

from langchain.globals import set_llm_cache
from langchain.schema import Generation
from langchain_openai import OpenAI, OpenAIEmbeddings
from langchain_redis import RedisCache, RedisSemanticCache

In [4]:
import langchain_core
import langchain_openai
import openai
import redis

### 设置 OpenAI API 密钥

In [ ]:
from getpass import getpass

# 检查环境中是否已设置 OPENAI_API_KEY
openai_api_key = os.getenv("OPENAI_API_KEY")

if not openai_api_key:
    print("在环境变量中未找到 OpenAI API 密钥。")
    openai_api_key = getpass("请输入您的 OpenAI API 密钥：")

    # 为当前会话设置 API 密钥
    os.environ["OPENAI_API_KEY"] = openai_api_key
    print("已为本次会话设置 OpenAI API 密钥。")
else:
    print("在环境变量中找到 OpenAI API 密钥。")

OpenAI API key not found in environment variables.


Please enter your OpenAI API key:  ········


OpenAI API key has been set for this session.


## 使用 RedisCache

In [ ]:
# 初始化 RedisCache
redis_cache = RedisCache(redis_url=REDIS_URL)

# 设置 LangChain 使用的缓存
set_llm_cache(redis_cache)

# 初始化语言模型
llm = OpenAI(temperature=0)


# 测量执行时间的函数
def timed_completion(prompt):
    start_time = time.time()
    result = llm.invoke(prompt)
    end_time = time.time()
    return result, end_time - start_time


# 第一次调用（未缓存）
prompt = "用三句话解释缓存的概念。"
result1, time1 = timed_completion(prompt)
print(f"第一次调用（未缓存）：\n结果：{result1}\n时间：{time1:.2f} 秒\n")

# 第二次调用（应该被缓存）
result2, time2 = timed_completion(prompt)
print(f"第二次调用（已缓存）：\n结果：{result2}\n时间：{time2:.2f} 秒\n")

print(f"速度提升：快了 {time1 / time2:.2f} 倍")

# 清除缓存
redis_cache.clear()
print("缓存已清除")

First call (not cached):
Result: 

Caching is the process of storing frequently accessed data in a temporary storage location for faster retrieval. This helps to reduce the time and resources needed to access the data from its original source. Caching is commonly used in computer systems, web browsers, and databases to improve performance and efficiency.
Time: 1.16 seconds

Second call (cached):
Result: 

Caching is the process of storing frequently accessed data in a temporary storage location for faster retrieval. This helps to reduce the time and resources needed to access the data from its original source. Caching is commonly used in computer systems, web browsers, and databases to improve performance and efficiency.
Time: 0.05 seconds

Speed improvement: 25.40x faster
Cache cleared


## 使用 RedisSemanticCache

In [ ]:
# 初始化 RedisSemanticCache
embeddings = OpenAIEmbeddings()
semantic_cache = RedisSemanticCache(
    redis_url=REDIS_URL, embeddings=embeddings, distance_threshold=0.2
)

# 设置 LangChain 使用的缓存
set_llm_cache(semantic_cache)


# 测试语义缓存的函数
def test_semantic_cache(prompt):
    start_time = time.time()
    result = llm.invoke(prompt)
    end_time = time.time()
    return result, end_time - start_time


# 原始查询
original_prompt = "法国的首都是什么？"
result1, time1 = test_semantic_cache(original_prompt)
print(
    f"原始查询：\n提示：{original_prompt}\n结果：{result1}\n时间：{time1:.2f} 秒\n"
)

# 语义相似的查询
similar_prompt = "你能告诉我法国的首都城市是什么吗？"
result2, time2 = test_semantic_cache(similar_prompt)
print(
    f"相似查询：\n提示：{similar_prompt}\n结果：{result2}\n时间：{time2:.2f} 秒\n"
)

print(f"速度提升：快了 {time1 / time2:.2f} 倍")

# 清除语义缓存
semantic_cache.clear()
print("语义缓存已清除")

Original query:
Prompt: What is the capital of France?
Result: 

The capital of France is Paris.
Time: 1.52 seconds

Similar query:
Prompt: Can you tell me the capital city of France?
Result: 

The capital of France is Paris.
Time: 0.29 seconds

Speed improvement: 5.22x faster
Semantic cache cleared


## 高级用法

### 自定义 TTL（生存时间）

In [ ]:
# 使用自定义 TTL 初始化 RedisCache
ttl_cache = RedisCache(redis_url=REDIS_URL, ttl=5)  # 60 秒 TTL

# 更新缓存条目
ttl_cache.update("test_prompt", "test_llm", [Generation(text="缓存的响应")])

# 检索缓存的条目
cached_result = ttl_cache.lookup("test_prompt", "test_llm")
print(f"缓存的结果：{cached_result[0].text if cached_result else '未找到'}")

# 等待 TTL 过期
print("等待 TTL 过期...")
time.sleep(6)

# 尝试检索过期的条目
expired_result = ttl_cache.lookup("test_prompt", "test_llm")
print(
    f"TTL 后的结果：{expired_result[0].text if expired_result else '未找到（已过期）'}"
)

Cached result: Cached response
Waiting for TTL to expire...
Result after TTL: Not found (expired)


### 自定义 RedisSemanticCache

In [ ]:
# 使用自定义设置初始化 RedisSemanticCache
custom_semantic_cache = RedisSemanticCache(
    redis_url=REDIS_URL,
    embeddings=embeddings,
    distance_threshold=0.1,  # 更严格的相似度阈值
    ttl=3600,  # 1 小时 TTL
    name="custom_cache",  # 自定义缓存名称
)

# 测试自定义语义缓存
set_llm_cache(custom_semantic_cache)

test_prompt = "太阳系中最大的行星是什么？"
result, _ = test_semantic_cache(test_prompt)
print(f"原始结果：{result}")

# 尝试略有不同的查询
similar_test_prompt = "太阳系中哪个行星最大？"
similar_result, _ = test_semantic_cache(similar_test_prompt)
print(f"相似查询结果：{similar_result}")

# 清理
custom_semantic_cache.clear()

Original result: 

The largest planet in our solar system is Jupiter.
Similar query result: 

The largest planet in our solar system is Jupiter.


## 结论

本笔记本演示了 langchain-redis 包中 `RedisCache` 和 `RedisSemanticCache` 的用法。这些缓存机制可以通过减少冗余的 API 调用并利用语义相似性进行智能缓存，显著提高基于 LLM 的应用程序的性能。基于 Redis 的实现为分布式系统中的缓存提供了快速、可扩展和灵活的解决方案。